In [1]:
# importuje niezbędne biblioteki
import re
from datetime import datetime
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os, sys
import glob
from matplotlib.backends.backend_pdf import PdfPages
import time

In [2]:
# Tworzymy listę i dwa puste DF
listOfFiles = []
DFL = pd.DataFrame()
DFM = pd.DataFrame() 

In [3]:
# Funkcja pobiera listę ścieżek
def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir


choosen_dir = get_filenames()
# glob zbiera listę plików 
os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)

# Do wcześniej stworzonych DF dodajemy kolejno pliki w zależności od tego na jaką literę plik się zaczyna.
for file in listOfFiles:
    if file.startswith('L'):
        Table = pd.read_csv(file, sep=',')
        DF1 = pd.DataFrame(Table)
        DFL = DFL.append(DF1, sort=False)
    elif file.startswith('M'):
        Table2 = pd.read_csv(file, sep=',')
        DF2 = pd.DataFrame(Table2)
        DFM = DFM.append(DF2, sort=False)

Initializing Dialogue... 
Please select a file.


In [4]:
# Deklaruje listę kolumn którę będą mi potrzebne
DFL['NMIOT'] = 1
cols1 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','NROJM1','LSUT1', 'NRMAM1', 'NOJMIO']
DFL = DFL[cols1]

In [5]:
# Zmieniam nazwy kolumn tak by odpowiadały kolumnon z DFL
DFM['LSUT1'] = np.nan  
DFM = DFM.rename(columns={'LDUR': 'LDUR1'})
DFM = DFM.rename(columns={'LDOP': 'LDUR'})
DFM = DFM.rename(columns={'LDUR1': 'LDOP'})
DFM = DFM.rename(columns={'RASAOJ': 'LRASAOJ'})
DFM = DFM.rename(columns={'NRO': 'NROJM1'})
DFM = DFM.rename(columns={'LREJ': 'LREJ1'})
DFM = DFM.rename(columns={'LWOJ': 'LWOJ1'})
# Deklaruje listę kolumn którę będą mi potrzebne
cols2 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','LSUT1','NROJM1']
DFM = DFM[cols2]

In [6]:
# Rozdzielam sutki prawe od lewych
DFL['PSUT'] = DFL['LSUT1'].astype(str).str[1]
DFL['LSUT1'] = DFL['LSUT1'].astype(str).str[0]

In [7]:
# Convertuje liczby sutków do inta
convert_dict = {'LSUT1': int,
                'PSUT': int
               }
  
DFL = DFL.astype(convert_dict)

In [8]:
# Liczę sumę sutków prawych i lewych
DFL['ISUT'] = DFL['PSUT'] + DFL['LSUT1']

In [9]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
date1 = input('Date range from YYYY-MM-DD') 
date2 = input('Date range to YYYY-MM-DD')

Date range from YYYY-MM-DD 2019-12-12
Date range to YYYY-MM-DD 2020-12-11


In [10]:
# Funkcja zamienia format kolumn na format daty
class ToDate():

    def change_to_date(self, a, b):
        a[b] = pd.to_datetime(a[b])


dat = ToDate()
dat.change_to_date(DFL, 'LDOP')
dat.change_to_date(DFM, 'LDOP')
dat.change_to_date(DFL, 'LDUR')
dat.change_to_date(DFM, 'LDUR')

In [11]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
DFL = DFL[(DFL['LDOP'] >= date1) & (DFL['LDOP'] <= date2)]
DFM = DFM[(DFM['LDOP'] >= date1) & (DFM['LDOP'] <= date2)]

In [12]:
# Wyliczamy:
# Datę pierwszego oprosienia
DFL['POP'] = DFL['LDOP'] - DFL['LDUR']
# Okres międzymiotu
DFM['OMM'] = DFM['LDOP'] - DFM['LDUR']
# Zamieniam ww wartości na dni
DFL['POP'] = DFL['POP'] / np.timedelta64(1, 'D')
DFM['OMM'] = DFM['OMM'] / np.timedelta64(1, 'D')

In [13]:
# W zmiennej DF3 łączymy poprzednie DF
DF3 = DFL.append(DFM)

In [14]:
DF3 = DF3.rename(columns={'LDUR': 'DURL'})
DF3 = DF3.rename(columns={'LDOP': 'DURM'})

In [15]:
# Wyliczamy ile prosiąt nie dożyło 21 dnia życia
DF3['STRATY'] = DF3['LIL11'] - DF3['LIL21']

In [16]:
# Wyliczamy wartość % strat
DF3['ST21'] = (DF3['STRATY'] / DF3['LIL11']) * 100

In [17]:
# Funkcja tworzy DF z określonymi już kolumnami
def createDT(name):
    temp = pd.DataFrame(columns = ['Rasa','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = name
    return temp

In [18]:
# Funkcja sortuje dane po odpowiedniej rasie
def sortedDT(DT, col, number):
    newDT = DT[DT[col] == number]
    return newDT
# Funkcja sortuje dane po rasie matki i rasie ojca
def sorted2DT(DT,number1,number2):
    newDT = DT[(DT.LRASA1 == number1) & (DT.LRASAOJ == number2)]
    return newDT

def months(DT, col, col2):
    newDT = DT[DT[col] != DT[col2]]
    return newDT

In [19]:
# Funkcja wylicza wszystkie niezbędne wartości i przypisuje je do DF
def all_count(DF3, new):
    new['liczba loch'] = len(DF3.drop_duplicates(subset='LPROS1', keep="first").copy())
    new['liczba ocenionych miotów ogółem'] = len(DF3)
    new['liczba ocenionych miotów pierw.'] = len(DF3[DF3['NMIOT']==1])
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DF3['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DF3['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DF3['ST21'].mean(), decimals=2)
    new['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(DF3['POP'].mean())
    new['Okres miedzymiotu(dni)'] = np.around(DF3['OMM'].mean())
    new['Liczba sutków lochy'] = np.around(DF3['ISUT'].mean(), decimals=2)

    return new

In [20]:
WBP = createDT('Wbp')
wbp = sortedDT(DF3,'LRASA1',10).copy()
all_count(wbp,WBP)

PBZ = createDT('Pbz')
pbz = sortedDT(DF3,'LRASA1',20).copy()
all_count(pbz,PBZ)

HAMP = createDT('Hampshire')
hamp = sortedDT(DF3,'LRASA1',60).copy()
all_count(hamp,HAMP)

DUR = createDT('Duroc')
dur = sortedDT(DF3,'LRASA1',70).copy()
all_count(dur,DUR)

PIET = createDT('Pietrain')
piet = sortedDT(DF3,'LRASA1',80).copy()
all_count(piet,PIET)

L99 = createDT('Linia 990')
l99 = sortedDT(DF3,'LRASA1',90).copy()
all_count(l99,L99)

ZB = createDT('Złotnicka Biała')
zb = sortedDT(DF3,'LRASA1',30).copy()
all_count(zb,ZB)

ZPS = createDT('Złotnicka Pstra')
zps = sortedDT(DF3,'LRASA1',50).copy()
all_count(zps,ZPS)

PULA = createDT('Puławska')
pula = sortedDT(DF3,'LRASA1',40).copy()
all_count(pula,PULA)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,Puławska,2110,3316,590,10.29,9.14,10.64,14.34,378.0,202.0


In [21]:
#Łącze wyniki ras w jeden DF i ustalam Rasę jako index
rasaDF = pd.concat([WBP,PBZ,HAMP,DUR,PIET,L99,PULA,ZB,ZPS])
rasaDF.set_index('Rasa',inplace=True)
rasaDF

,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
Rasa,,,,,,,,,
Wbp,1944,3315,736,12.48,11.65,6.26,14.99,365.0,167.0
Pbz,3104,5134,1287,12.65,11.72,6.82,15.29,359.0,173.0
Hampshire,13,26,5,12.42,11.04,11.15,14.20,445.0,182.0
Duroc,328,556,103,11.87,10.82,8.26,13.66,371.0,180.0
Pietrain,298,522,95,11.58,10.90,5.54,14.11,383.0,181.0
Linia 990,895,1580,335,9.96,9.48,4.78,14.19,374.0,155.0
Puławska,2110,3316,590,10.29,9.14,10.64,14.34,378.0,202.0
Złotnicka Biała,1134,1627,208,8.62,8.00,6.44,14.00,384.0,235.0
Złotnicka Pstra,998,1272,167,8.24,6.98,14.56,13.24,426.0,262.0


In [39]:
# Deklaruje słoknik miast
dic_miasta = {1:'Poznań',
             2:'Warszawa',
             3:'Poznań',
             4:'Białystok',
             6:'Bydgoszcz',
             8:'Gdańsk',
             10:'Katowice',
             12:'Kielce',
             14:'Koszalin',
             16:'Kraków',
             18:'Lublin',
             20:'Łódź',
             22:'Olsztyn',
             24:'Opole',
             26:'Poznań',
             28:'Rzeszów',
             32:'Wrocław',
             34:'Zielona Góra',
             98:'98'}
# Deklaruje listę wydzielonych ras
race_list = [wbp,pbz,hamp,dur,piet,l99,zb,zps,pula]
race_names = ['WBP','PBZ','Hampshire','Duroc','Pietrain','Linia 990','PUL','ZLB','ZLP']

In [24]:
def create_cityDT(city, race):
    temp = pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Miasto'] = city
    if race is wbp:
        temp['Rasa'] = "WBP"
    elif race is pbz:
        temp['Rasa'] = "PBZ"
    elif race is hamp:
        temp['Rasa'] = "Hampshire"
    elif race is dur:
        temp['Rasa'] = "Duroc"
    elif race is piet:
        temp['Rasa'] = "Pietrain"
    elif race is l99:
        temp['Rasa'] = "Linia 99"
    elif race is zb:
        temp['Rasa'] = "Złotnicka Biała"
    elif race is zps:
        temp['Rasa'] = "Złotnicka Pstra"
    elif race is pula:
        temp['Rasa'] = "Puławska"
    return temp

cityDF = pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])

temp =  pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])



In [33]:
CompleteCityDF = pd.DataFrame()

for city in dic_miasta:
    for race in race_list:
        tempDF = create_cityDT(dic_miasta[city],race)    
        tempDF2 = sortedDT(race, "LREJ1", city)
        rdy = all_count(tempDF2,tempDF)
        CompleteCityDF = pd.concat([CompleteCityDF,rdy])

In [35]:
CompleteCityDF['Okres miedzymiotu(dni)'] = CompleteCityDF['Okres miedzymiotu(dni)'].fillna('-')
CompleteCityDF = CompleteCityDF.set_index(['Rasa','Miasto']).sort_index(ascending = True).dropna()

In [37]:
CompleteCityDF

liczba loch  liczba ocenionych miotów ogółem  \
Rasa            Miasto                                                       
Duroc           98                     44                               68   
                Bydgoszcz             119                              218   
                Gdańsk                  7                               11   
                Olsztyn                49                               78   
                Poznań                 73                              124   
                Warszawa               14                               20   
                Łódź                   22                               37   
Hampshire       Bydgoszcz              13                               26   
Linia 99        Poznań                895                             1580   
PBZ             98                    340                              585   
                Białystok               5                                6   
                Bydgoszcz             843                             1485   
                Gdańsk                300                              499   
                Katowice              125                              204   
                Kielce                 91                              148   
                Kraków                 52                               79   
                Lublin                 21                               40   
                Olsztyn               153                              256   
                Opole                 108                              158   
                Poznań                663                             1013   
                Rzeszów                47                               77   
                Warszawa              140                              232   
                Zielona Góra           22                               49   
                Łódź                  194                              303   
Pietrain        Bydgoszcz              72                              133   
                Gdańsk                  9                               12   
                Katowice               10                               19   
                Olsztyn               118                              212   
                Opole                  43                               72   
                Poznań                 34                               58   
                Łódź                   12                               16   
Puławska        Białystok             132                              187   
                Bydgoszcz             314                              520   
                Gdańsk                115                              210   
                Kielce                 15                               19   
                Koszalin               33                               67   
                Kraków                 29                               48   
                Lublin                951                             1523   
                Olsztyn                12                               29   
                Opole                  78                              117   
                Poznań                 17                               17   
                Warszawa              387                              551   
                Łódź                   21                               21   
WBP             98                    393                              632   
                Białystok              53                              104   
                Bydgoszcz             656                             1178   
                Gdańsk                 79                              113   
                Lublin                 79                              129   
                Olsztyn                58                              106   
                Poznań                400           